<a href="https://colab.research.google.com/github/JituShinde77/ABSA_Project/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Credits: https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
# LSTM for sequence classification in the IMDB dataset

import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import pandas as pd

Using TensorFlow backend.


In [0]:
# Way to load CSV file in google colab https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

     |████████████████████████████████| 993kB 39.6MB/s 


In [0]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# https://stackoverflow.com/questions/49685924/extract-google-drive-zip-from-google-colab-notebook
import zipfile
import os
from google.colab import drive
print (os.getcwd())
drive.mount('/content/drive/')



/content
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
#!unzip '/content/drive/My Drive/LSTM Assignment/amazon-fine-food-reviews.zip' -d '/content/drive/My Drive/LSTM Assignment/'

Archive:  /content/drive/My Drive/LSTM Assignment/amazon-fine-food-reviews.zip
  inflating: /content/drive/My Drive/LSTM Assignment/Reviews.csv  
  inflating: /content/drive/My Drive/LSTM Assignment/database.sqlite  
  inflating: /content/drive/My Drive/LSTM Assignment/hashes.txt  


In [0]:
link = "https://drive.google.com/open?id=1-1yJpRCJdZk-XddvzHXHT8MmErBMCPU0"
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1-1yJpRCJdZk-XddvzHXHT8MmErBMCPU0


In [0]:
import sqlite3
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('database.sqlite')  
con = sqlite3.connect('database.sqlite') 
# Dataset is now stored in a Pandas Dataframe


W0901 15:45:52.762044 140167846676352 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import Lock

In [0]:
filtered_data = pd.read_sql_query("""SELECT * FROM Reviews WHERE Score != 3 LIMIT 50000""", con)

def partition(x):
  if x < 3:
    return 0
  return 1

actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition)
filtered_data['Score'] = positiveNegative



print ("Number of Data Points",filtered_data.shape)
filtered_data.head(3)



Number of Data Points (50000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [0]:
#Sorting data according to ProductId in ascending order
sorted_data = filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='qicksort', na_position='last' )

In [0]:
#Deduplication of entries
final = sorted_data.drop_duplicates(subset={'UserId', 'ProfileName', 'Time', 'Text'}, keep='first', inplace=False)
final.shape

(46072, 10)

In [0]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0) * 100

92.144

In [0]:
# Keep Id text
X = final.drop(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator','HelpfulnessDenominator', 'Time', 'Summary'], axis=1)

In [0]:

y = X['Score'].values
X.drop(['Score'], axis=1, inplace = True)
X.head(5)

,Text
22620,My dogs loves this chicken but its a product f...
22621,Our dogs just love them. I saw them in a pet ...
2546,Why is this $[...] when the same product is av...
2547,We have used the Victor fly bait for 3 seasons...
1145,I just received my shipment and could hardly w...


In [0]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [0]:
# Train and test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify= y)


In [0]:
# Combining all the above stundents 
from tqdm import tqdm
X_train_preprocessed_text = []
# tqdm is for printing the status bar
for sentance in tqdm(X_train['Text'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    
    X_train_preprocessed_text.append(sent.lower().strip())

100%|██████████| 30868/30868 [00:00<00:00, 31195.46it/s]


In [0]:
# Combining all the above stundents 
from tqdm import tqdm
X_test_preprocessed_text = []
# tqdm is for printing the status bar
for sentance in tqdm(X_test['Text'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    
    X_test_preprocessed_text.append(sent.lower().strip())

100%|██████████| 15204/15204 [00:00<00:00, 32051.43it/s]


In [0]:
from tqdm import tqdm
X_test_preprocessed_text = []
# tqdm is for printing the status bar
for sentance in tqdm(X_test['Text'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    
    X_test_preprocessed_text.append(sent.lower().strip())

100%|██████████| 15204/15204 [00:00<00:00, 32514.07it/s]


In [0]:
vocabulary = []
for sentence in tqdm(X_train_preprocessed_text):
  vocabulary.extend(sentence.split(" "))
  
print ( len(vocabulary))

100%|██████████| 30868/30868 [00:00<00:00, 125283.67it/s]

2511585


In [0]:
print (len(vocabulary))

2511585


In [0]:
#https://programminghistorian.org/en/lessons/counting-frequencies
# Ordered Set https://pypi.org/project/ordered-set/
!pip install ordered_set
from collections import Counter

from ordered_set import OrderedSet
print (len(vocabulary))
wordset = OrderedSet(vocabulary)

print (len(wordset))
wordfreq = []
for w in tqdm(wordset):
    wordfreq.append(vocabulary.count(w))
    
wordfreq[0]


  Created wheel for ordered-set: filename=ordered_set-3.1.1-py2.py3-none-any.whl size=7813 sha256=1891eb2d6fab29bfb22ec783a6f8d868a50c1c024baa7ee0e1fc89149ba19a3e
  Stored in directory: /root/.cache/pip/wheels/dd/70/f1/0a4953bd6d06e7dbffe8ef0197b558eaecb21fa6b043b50d73
Successfully built ordered-set
2511585


  0%|          | 3/33700 [00:00<20:01, 28.06it/s]

33700


100%|██████████| 33700/33700 [19:41<00:00, 28.53it/s]


89283

In [0]:

wordfreq = numpy.array(list(wordfreq))
sortIndex = numpy.argsort(list(-wordfreq)) 
wordlist = numpy.array(list(wordset))
sortWords = [wordlist[i] for i in list(numpy.argsort(list(-wordfreq)))]

vocIndex = dict(zip(sortWords, list(range(1, len(wordfreq)+1))))
print(vocIndex)


{'the': 1, 'i': 2, 'and': 3, 'a': 4, 'it': 5, 'is': 6, 'to': 7, 'of': 8, 'this': 9, 'br': 10, 'not': 11, 'for': 12, 'in': 13, 'my': 14, 'have': 15, 'that': 16, 'you': 17, 'are': 18, 'but': 19, 'with': 20, 'was': 21, 'they': 22, 'on': 23, 'as': 24, 'so': 25, 'like': 26, 'these': 27, 'them': 28, 'good': 29, 'be': 30, 'at': 31, 'can': 32, 'or': 33, 'one': 34, 'will': 35, 'great': 36, 'just': 37, 'very': 38, 'taste': 39, 'product': 40, 'do': 41, 'if': 42, 'all': 43, 'would': 44, 'we': 45, 'coffee': 46, 'from': 47, 'am': 48, 'tea': 49, 'flavor': 50, 'when': 51, 'has': 52, 'love': 53, 'more': 54, 'had': 55, 'me': 56, 'food': 57, 'out': 58, 'than': 59, 'no': 60, 'amazon': 61, 'get': 62, 'other': 63, 'up': 64, 'about': 65, 'some': 66, 'only': 67, 'really': 68, 'too': 69, 'an': 70, 'much': 71, 'your': 72, 'time': 73, 'also': 74, 'there': 75, 'use': 76, 'because': 77, 'he': 78, 'were': 79, 'best': 80, 'little': 81, 'chocolate': 82, 'she': 83, 'been': 84, 'tried': 85, 'buy': 86, 'price': 87, 'wha

In [0]:

X_train = []

for sentence in tqdm(X_train_preprocessed_text):
  words = sentence.split(" ")
  words_count = []
  for word  in words:
    if (vocIndex[word] < 5000):
      words_count.append(vocIndex[word])
  X_train.append(words_count)
X_train = numpy.array(X_train)

100%|██████████| 30868/30868 [00:00<00:00, 33712.29it/s]


In [0]:
X_test = []
cnt = 0

for sentence in tqdm(X_test_preprocessed_text):
  words = sentence.split(" ")
  words_count = []
  for word  in words:
    try:
      if (vocIndex[word] < 5000):
        words_count.append(vocIndex[word])
    except:
      cnt +=1
  
  X_test.append(words_count)
X_test = numpy.array(X_test)
print ("\n Number of words not present in test data", cnt)

100%|██████████| 15204/15204 [00:00<00:00, 21547.98it/s]


 Number of words not present in test data 8109


In [0]:
X_train.shape

(30868,)

In [0]:
# Truncat add/or pad input sequence

max_review_length = 600
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

print (X_train.shape)
print (X_train[1])

(30868, 600)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    

In [0]:
# Create LSTM Model
embedding_vector_length = 32

model = Sequential()
model.add(Embedding(5000+1, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.summary()



Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 600, 32)           160032    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 213,333
Trainable params: 213,333
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [0]:
model.fit(X_train, y_train, nb_epoch=10, batch_size=64)

scores = model.evaluate(X_test, y_test, verbose=0)
print ("Accuracy: %.2f%%" % (scores[1]*100))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/10
30868/30868 [==============================] - 568s 18ms/step - loss: 0.2996 - acc: 0.8819
Epoch 2/10
30868/30868 [==============================] - 567s 18ms/step - loss: 0.1899 - acc: 0.9274
Epoch 3/10
30868/30868 [==============================] - 566s 18ms/step - loss: 0.1627 - acc: 0.9385
Epoch 4/10
30868/30868 [==============================] - 565s 18ms/step - loss: 0.1426 - acc: 0.9476
Epoch 5/10
30868/30868 [==============================] - 566s 18ms/step - loss: 0.1300 - acc: 0.9511
Epoch 6/10
30868/30868 [==============================] - 565s 18ms/step - loss: 0.1116 - acc: 0.9596
Epoch 7/10
30868/30868 [==============================] - 562s 18ms/step - loss: 0.0966 - acc: 0.9653
Epoch 8/10
30868/30868 [==============================] - 562s 18ms/step - loss: 0.1033 - acc: 0.9646
Epoch 9/10
30868/30868 [==============================] - 569s 18ms/step - loss: 0.0911 - acc: 0.9682
Epoch 10/10
30868/30868 [==============================] - 570s 18ms/step - loss: 

# **Second Model**
2 LSTM

In [0]:
# https://stackoverflow.com/questions/40331510/how-to-stack-multiple-lstm-in-keras
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(5000+1, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
#Refer: https://datascience.stackexchange.com/questions/10615/number-of-parameters-in-an-lstm-model

W0901 16:09:42.134938 140167846676352 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0901 16:09:42.178879 140167846676352 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0901 16:09:42.185181 140167846676352 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0901 16:09:42.606882 140167846676352 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0901 16:09:42.626441 140167846676352 deprecation_wrappe

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 600, 32)           160032    
_________________________________________________________________
lstm_1 (LSTM)                (None, 600, 100)          53200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 243,483
Trainable params: 243,483
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model.fit(X_train, y_train, nb_epoch=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.
W0901 16:09:48.211841 140167846676352 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/10
30868/30868 [==============================] - 885s 29ms/step - loss: 0.2963 - acc: 0.8823
Epoch 2/10
30868/30868 [==============================] - 885s 29ms/step - loss: 0.1946 - acc: 0.9240
Epoch 3/10
30868/30868 [==============================] - 880s 29ms/step - loss: 0.1697 - acc: 0.9362
Epoch 4/10
30868/30868 [==============================] - 899s 29ms/step - loss: 0.1629 - acc: 0.9378
Epoch 5/10
30868/30868 [==============================] - 894s 29ms/step - loss: 0.1391 - acc: 0.9468
Epoch 6/10
30868/30868 [==============================] - 881s 29ms/step - loss: 0.1190 - acc: 0.9565
Epoch 7/10
30868/30868 [==============================] - 878s 28ms/step - loss: 0.1055 - acc: 0.9624
Epoch 8/10
30868/30868 [==============================] - 884s 29ms/step - loss: 0.1043 - acc: 0.9614
Epoch 9/10
30868/30868 [==============================] - 877s 28ms/step - loss: 0.0910 - acc: 0.9676
Epoch 10/10
30868/30868 [==============================] - 877s 28ms/step - loss: 

# **Conclusion**

In [0]:
# Please compare all your models using Prettytable library

from prettytable import PrettyTable


x = PrettyTable()
x.field_names = ["Number of LSTM Layers", "Layers",  "Accuracy"]

x.add_row(["1",  "LSTM(100) ", 90.92])
x.add_row(["....................",  "........ ", "......"])
x.add_row(["2",  "LSTM(100) \n LSTM(50)", 90.75])

print(x)

+-----------------------+------------+----------+
| Number of LSTM Layers |   Layers   | Accuracy |
+-----------------------+------------+----------+
|           1           | LSTM(100)  |  90.92   |
|  .................... | ........   |  ......  |
|           2           | LSTM(100)  |  90.75   |
|                       |  LSTM(50)  |          |
+-----------------------+------------+----------+
